In [19]:
API_URL = "https://librivox.org/search/get_results?primary_key=71&search_category=genre&sub_category=&search_page={page}&search_order=alpha&project_type=either"
HEADERS = {
    "X-Requested-With": "XMLHttpRequest",
}

In [28]:
import requests
import json
from bs4 import BeautifulSoup

In [47]:
def process_search_page(page=1):
    url = API_URL.format(page=page)
    req = requests.get(url, headers=HEADERS)
    assert req.status_code == 200
    res = json.loads(req.text)
    assert "results" in res
    soup = BeautifulSoup(res["results"], "html.parser")
    links = []
    for li in soup.find_all("li", {"class": "catalog-result"}):
        for resdata in li.find_all("div", {"class": "result-data"}):
            a = [x for x in resdata.find_all("a")]
            filtered = []
            for link in a:
                # if not "href" in link:
                #     continue
                if not "/author/" in link["href"]:
                    filtered.append(link["href"])
            if len(filtered) != 1:
                print("Error", filtered)
            links += filtered
    return links


In [50]:
def do_searches(links=[], search_page=1):
    current = process_search_page(search_page)
    stack = links + current
    if len(current) != 25:
        return stack
    else:
        return do_searches(stack, search_page+1)

In [51]:
pages = do_searches([], 1)

In [54]:
with open("pages.txt", "w") as outf:
    for page in pages:
        outf.write(page + "\n")